In [1]:
import csv
import json
import requests
import re
from pathlib import Path

In [5]:
# define model, ollama API, input files
model_name = "phi4-reasoning:plus" # model name here
ollama_url = "http://localhost:11434/api/generate"

input_files = [
    "relevance_210725_prompts_templ-1.csv",
    "relevance_210725_prompts_templ-2.csv",
    "relevance_210725_prompts_templ-3.csv",
    "relevance_210725_prompts_templ-4.csv",
    "relevance_210725_prompts_templ-5.csv",
]

In [ ]:
# after many-a trial, thisscript finally works
# GK NOTE 23 JULY: Saw script run supervised for templ-1, now leaving to run templ 2-5
import time
import json
import requests
import re
from pathlib import Path
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging
from typing import List, Dict, Any

# Configuration
MAX_WORKERS = 12  # Increased workers for slow model
TEST_SUBSET = None # None to process all rows
REQUEST_TIMEOUT = 150# Longer timeout but with aggressive parallelization
RETRY_ATTEMPTS = 1  # Reduced retries to fail fast

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class OllamaProcessor:
    """Optimized Ollama API processor with yes/no normalization."""
    
    def __init__(self, url: str, model: str, max_workers: int = MAX_WORKERS):
        self.url = url
        self.model = model
        self.max_workers = max_workers
        self.session = requests.Session()
        self.session.headers.update({"Content-Type": "application/json"})
        adapter = requests.adapters.HTTPAdapter(
            pool_connections=max_workers,
            pool_maxsize=max_workers * 2,
            max_retries=0
        )
        self.session.mount("http://", adapter)
        self.session.mount("https://", adapter)
        
    def create_payload(self, prompt: str) -> Dict[str, Any]:
        return {
            "model": self.model,
            "prompt": prompt.strip(),
            "stream": False,
            "options": {
                "temperature": 0.0,
                "top_p": 0.1,
                "num_predict": -1,
                "num_ctx": 4096,
                "repeat_penalty": 1.0
            }
        }
    
    def normalize_response(self, text: str) -> str:
        """Strip <think>…</think>, then normalize to Yes/No if response ends with them."""
        if not text:
            return "Error: Empty response"
        text = text.strip()
        # Remove any <think>...</think> block
        m = re.search(r'<think>.*?</think>\s*(.*)', text, re.IGNORECASE | re.DOTALL)
        if m:
            text = m.group(1).strip()

        low = text.lower().rstrip('.!?')  # drop any trailing punctuation
        if low.endswith("yes"):
            return "Yes"
        if low.endswith("no"):
            return "No"
        return text  # fallback to full text

    
    def extract_response_text(self, result: Any) -> str:
        """Extract text from various Ollama JSON shapes."""
        if isinstance(result, dict):
            for key in ("response", "output", "content"):
                if key in result and isinstance(result[key], str):
                    return result[key].strip()
            # Chat-style
            if "choices" in result and isinstance(result["choices"], list):
                msg = result["choices"][0].get("message", {})
                return msg.get("content", "").strip()
            if "message" in result and isinstance(result["message"], dict):
                return result["message"].get("content", "").strip()
            return str(result)
        if isinstance(result, list) and result:
            return str(result[0])
        return str(result)

    def call_ollama_single(self, prompt: str) -> str:
        if not prompt:
            return "Error: Empty prompt"
        payload = self.create_payload(prompt)
        for attempt in range(RETRY_ATTEMPTS + 1):
            try:
                resp = self.session.post(self.url, json=payload, timeout=REQUEST_TIMEOUT)
                if resp.status_code == 200:
                    try:
                        result = resp.json()
                        text = self.extract_response_text(result)
                        return self.normalize_response(text)
                    except json.JSONDecodeError as e:
                        return f"JSONError: {e}"
                else:
                    return f"HTTPError: {resp.status_code}"
            except requests.exceptions.Timeout:
                return "TimeoutError: Request timed out"
            except requests.exceptions.ConnectionError as e:
                return f"ConnectionError: {e}"
            except Exception as e:
                return f"UnexpectedError: {e}"
        return "Error: All retry attempts failed"
    
    def process_all_parallel(self, data_items: List[tuple]) -> List[tuple]:
        """Process items in parallel and report accurate ETA."""
        total = len(data_items)
        completed = 0
        results: List[tuple] = []
        start = time.perf_counter()
        
        logger.info(f"Starting processing of {total} items with {self.max_workers} workers")
        with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            future_to_idx = {executor.submit(self.call_ollama_single, prompt): idx
                             for idx, prompt in data_items}
            
            for future in as_completed(future_to_idx):
                idx = future_to_idx[future]
                try:
                    res = future.result()
                except Exception as e:
                    res = f"FutureError: {e}"
                    logger.error(f"Error in future for row {idx}: {e}")
                results.append((idx, res))
                completed += 1

                if completed % 5 == 0 or completed == total:
                    elapsed = time.perf_counter() - start
                    avg = elapsed / completed
                    remaining = total - completed
                    eta_sec = remaining * avg
                    eta_min = eta_sec / 60
                    logger.info(
                        f"Progress: {completed}/{total} ({100*completed/total:.1f}%) | "
                        f"Avg: {avg:.1f}s | ETA: {eta_min:.1f}min"
                    )

        return results

def process_file_optimized(input_path: str, processor: OllamaProcessor):
    match = re.search(r"templ-\d+", input_path)
    if not match:
        logger.warning(f"Skipping {input_path}, no template tag found.")
        return
    template_tag = match.group()
    model_tag = processor.model.replace(":", "-")
    output_path = f"relevance_210725_completions_{model_tag}-{template_tag}.csv"
    
    logger.info(f"Processing {input_path} -> {output_path}")
    try:
        df = pd.read_csv(input_path, encoding='utf-8')
    except Exception as e:
        logger.error(f"Failed to read {input_path}: {e}")
        return
    if df.empty:
        logger.error(f"No data in {input_path}")
        return
    
    # Ensure eval_prompt column
    if 'eval_prompt' not in df.columns:
        df.rename(columns={df.columns[-1]: 'eval_prompt'}, inplace=True)
    
    # changed to None to run through templ-1 file
    if TEST_SUBSET and len(df) > TEST_SUBSET:
        df = df.head(TEST_SUBSET).copy()
        logger.info(f"TEST MODE: Processing first {len(df)} rows")
    
    df['eval_completion'] = None
    df['model'] = processor.model
    
    valid = df['eval_prompt'].notna() & (df['eval_prompt'] != "")
    valid_indices = df[valid].index.tolist()
    if not valid_indices:
        logger.error("No valid prompts to process.")
        return
    
    # Parallel processing
    data = [(idx, df.at[idx, 'eval_prompt']) for idx in valid_indices]
    t0 = time.perf_counter()
    results = processor.process_all_parallel(data)
    total_time = time.perf_counter() - t0
    
    # Update DataFrame
    for idx, completion in results:
        df.at[idx, 'eval_completion'] = completion
    
    # Summary stats
    successful = df['eval_completion'].notna().sum()
    errors = df['eval_completion'].str.contains('Error:', na=False).sum()
    timeouts = df['eval_completion'].str.contains('TimeoutError:', na=False).sum()
    rate = 100 * (successful - errors) / len(df)
    
    logger.info("="*50)
    logger.info(f"Finished in {total_time/60:.2f} minutes")
    logger.info(f"Rows processed: {len(df)} (Valid: {len(valid_indices)})")
    logger.info(f"Successes: {successful - errors}, Errors: {errors} (Timeouts: {timeouts})")
    logger.info(f"Success rate: {rate:.1f}%")
    logger.info(f"Throughput: {len(valid_indices)/(total_time/60):.1f} req/min")
    logger.info("="*50)
    
    # Save
    try:
        df.to_csv(output_path, index=False, encoding='utf-8')
        logger.info(f"Saved to {output_path}")
    except Exception as e:
        logger.error(f"Failed to save {output_path}: {e}")

def main():
    logger.info("Starting optimized processing")
    logger.info(f"- Model: {model_name}")
    logger.info(f"- Ollama URL: {ollama_url}")
    logger.info(f"- Max workers: {MAX_WORKERS}")
    if TEST_SUBSET:
        logger.info(f"- TEST SUBSET: {TEST_SUBSET} rows")
    
    processor = OllamaProcessor(ollama_url, model_name, MAX_WORKERS)
    
    for input_path in input_files:
        if not Path(input_path).exists():
            logger.error(f"File not found: {input_path}")
            continue
        process_file_optimized(input_path, processor)

if __name__ == "__main__":
    main()


# one file took 184 minutes, only 4.6% timeout error


2025-07-24 10:29:26,207 - INFO - Starting optimized processing
2025-07-24 10:29:26,208 - INFO - - Model: phi4-reasoning:plus
2025-07-24 10:29:26,209 - INFO - - Ollama URL: http://localhost:11434/api/generate
2025-07-24 10:29:26,210 - INFO - - Max workers: 12
2025-07-24 10:29:26,211 - INFO - Processing relevance_210725_prompts_templ-1.csv -> relevance_210725_completions_phi4-reasoning-plus-templ-1.csv
2025-07-24 10:29:26,225 - INFO - Starting processing of 1000 items with 12 workers
2025-07-24 10:31:56,322 - INFO - Progress: 5/1000 (0.5%) | Avg: 30.0s | ETA: 497.8min
2025-07-24 10:31:56,362 - INFO - Progress: 10/1000 (1.0%) | Avg: 15.0s | ETA: 247.7min


In [ ]:
# original, took to long

for input_path in input_files:
    # Determine output file name based on template number and model
    template_match = re.search(r"templ-\d+", input_path)
    if not template_match:
        continue
    template_tag = template_match.group() 
    # replace colon with hyphen in models
    model_tag = model_name.replace(":", "-")
    output_path = f"relevance_210725_completions_{model_tag}-{template_tag}.csv"
    
    with open(input_path, newline='', encoding='utf-8') as infile, \
         open(output_path, 'w', newline='', encoding='utf-8') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        
        # Read the header and append new column names
        header = next(reader)
        new_header = header + ["eval_completion", "model"]
        writer.writerow(new_header)
        
        # Iterate over each row in the input CSV
        for row in reader:
            if not row:  # skip empty lines if any
                continue
            prompt = row[-1]  # eval_prompt is the last col
            
            # Prepare the JSON payload for Ollama API
            payload = {
                "model": model_name,
                "prompt": prompt,
                "stream": False  # get a single JSON response instead of stream
            }
            
            try:
                response = requests.post(ollama_url, headers={"Content-Type": "application/json"},
                                         data=json.dumps(payload))
            except Exception as e:
                # If there's a connection error or similar, you might want to handle it
                print(f"Error calling Ollama API for prompt: {prompt[:30]}... \n{e}")
                continue
            
            eval_completion = ""
            if response.status_code == 200:
                # Parse JSON response from Ollama
                try:
                    result = response.json()
                except ValueError:
                    # If response is not a valid JSON (unexpected), use raw text
                    result_text = response.text.strip()
                    # Determine yes/no from text
                    if result_text.lower().startswith("yes"):
                        eval_completion = "Yes"
                    elif result_text.lower().startswith("no"):
                        eval_completion = "No"
                    else:
                        eval_completion = result_text  # fallback to whatever it is
                else:
                    # Ollama's response JSON might have the output text in a field.
                    # We attempt common possible keys.
                    if "response" in result:
                        result_text = result["response"]
                    elif "output" in result:
                        result_text = result["output"]
                    elif "content" in result:
                        # If using chat-style response, it might be nested:
                        # e.g., {"model": ..., "choices": [{"message": {"role": "assistant", "content": "Yes"}}], ...}
                        result_text = result.get("content", "") or result.get("message", {}).get("content", "")
                    else:
                        # If none of the known keys, use the full JSON string as fallback
                        result_text = str(result)
                    result_text = str(result_text).strip()
                    # Normalize to "Yes" or "No"
                    if result_text.lower().startswith("yes"):
                        eval_completion = "Yes"
                    elif result_text.lower().startswith("no"):
                        eval_completion = "No"
                    else:
                        eval_completion = result_text
            else:
                # If the API call failed (non-200 status), record the status or an error
                eval_completion = f"Error: HTTP {response.status_code}"
            
            # Append the new columns to the row
            row_with_output = row + [eval_completion, model_name]
            writer.writerow(row_with_output)

    print(f"Completed {input_path} -> {output_path}")